In [ ]:
from math import ceil, exp
from os.path import join
from typing import Callable
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt

# plt.rc("axes", labelsize=18)
# plt.rc("axes", titlesize=18)
plt.rc(
    "font", size=18
)  # controls default text sizes   # fontsize of the x and y labels
# plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
# plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
# plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
# plt.rc('figure', titlesize=BIGGER_SIZE)

In [ ]:
data = loadmat("carte_centreMetres.mat")
h_mat, x_val, y_val = data["h_MNT"], data["x_MNT"][0], data["y_MNT"][0]

In [ ]:
def h_mnt(
    x_val: np.ndarray,
    y_val: np.ndarray,
    h_mat: np.ndarray,
    x_float: float,
    y_float: float,
) -> float:
    x_floor, y_floor = (
        np.searchsorted(x_val, x_float) - 1,
        np.searchsorted(y_val, y_float) - 1,
    )
    if x_floor >= 798 or y_floor >= 798:
        print(x_float, y_float)
    alpha_x = (x_float - x_val[x_floor]) / (x_val[x_floor + 1] - x_val[x_floor])
    interp_y_floor = (
        alpha_x * h_mat[x_floor + 1, y_floor] + (1 - alpha_x) * h_mat[x_floor, y_floor]
    )
    interp_y_ceil = (
        alpha_x * h_mat[x_floor + 1, y_floor + 1]
        + (1 - alpha_x) * h_mat[x_floor, y_floor + 1]
    )
    alpha_y = (y_float - y_val[y_floor]) / (y_val[y_floor + 1] - y_val[y_floor])
    return alpha_y * interp_y_ceil + (1 - alpha_y) * interp_y_floor


def currified_h_mnt(
    x_val: np.ndarray, y_val: np.ndarray, h_mat: np.ndarray
) -> Callable[[float, float], float]:
    def currified(x_float: float, y_float: float) -> float:
        return h_mnt(
            x_val=x_val, y_val=y_val, h_mat=h_mat, x_float=x_float, y_float=y_float
        )

    return currified

In [ ]:
DT = 1
PHI = np.array(
    [
        [1, 0, 0, DT, 0, 0],
        [0, 1, 0, 0, DT, 0],
        [0, 0, 1, 0, 0, DT],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
    ]
)
CAP_AVION = 45 * np.pi / 180
V_AVION = 300 * 1000 / 3600
LONG_TRAJET = 60 * 1000
NB_IT = ceil(LONG_TRAJET / (DT * V_AVION))
VX = V_AVION * np.cos(CAP_AVION)
VY = V_AVION * np.sin(CAP_AVION)
ETAT_INIT = np.array([[150000], [150000], [8000], [VX], [VY], [0]])
SIGMA_BRUIT = 30
SIGMA_X = 3000
SIGMA_Y = 3000
SIGMA_Z = 500
SIGMA_VX = SIGMA_VY = SIGMA_VZ = 5
P_0 = np.diag(
    [
        SIGMA_X**2,
        SIGMA_Y**2,
        SIGMA_Z**2,
        SIGMA_VX**2,
        SIGMA_VY**2,
        SIGMA_VZ**2,
    ]
)
N_PART = 5000
x_avion = ETAT_INIT[0] + VX * DT * np.array(range(NB_IT))
y_avion = ETAT_INIT[1] + VY * DT * np.array(range(NB_IT))
z_avion = ETAT_INIT[2] * np.ones(shape=(NB_IT,))
etat_avion = np.array([x_avion, y_avion, z_avion])

In [ ]:
rd = np.random.default_rng(20)
epsilon_bruit = rd.normal(loc=0, scale=SIGMA_BRUIT, size=NB_IT)
h_mes = (
    z_avion
    - np.array(
        [
            currified_h_mnt(x_val=x_val, y_val=y_val, h_mat=h_mat)(
                x_float=x, y_float=y
            )  # type: ignore
            for (x, y) in zip(x_avion, y_avion)
        ]
    )
    + epsilon_bruit
)
epsilon: np.ndarray = rd.multivariate_normal(
    mean=6 * [0], cov=P_0, size=N_PART
).reshape((N_PART, 6, 1))
init_particles = ETAT_INIT + epsilon

In [ ]:
DIM_STATE = 6


def particle_filter(
    h_mnt: Callable[[float, float], float],
    h_mes: np.ndarray,
    init_particles: np.ndarray,
    phi: np.ndarray,
    sigma_bruit: float,
):
    nb_it = h_mes.shape[0]
    n_part = init_particles.shape[0]
    particles = np.empty(shape=(nb_it, n_part, DIM_STATE, 1))
    x_mean = np.empty(shape=(nb_it, DIM_STATE, 1))
    h_opt = (4 / (DIM_STATE + 2)) ** (1 / (DIM_STATE + 4)) * n_part ** (
        -1 / (DIM_STATE + 4)
    )
    h_fin = 0.4 * h_opt
    n_redis = 0.4 * n_part
    part_pred = np.empty(shape=(n_part, 6, 1))
    part_pred[:, :, :] = init_particles
    weights = np.empty(shape=(nb_it, n_part, 1, 1))
    weight_pred = np.empty(shape=(n_part, 1, 1))
    weight_pred[:, :, :] = 1 / n_part
    cov_mats = np.empty(shape=(nb_it, DIM_STATE, DIM_STATE))
    redis_moments = np.zeros(shape=(nb_it,))
    for k in range(nb_it):
        for i in range(n_part):
            ecart = h_mes[k] - (
                part_pred[i, 2, 0] - h_mnt(part_pred[i, 0, 0], part_pred[i, 1, 0])
            )
            weights[k, i, 0, 0] = weight_pred[i, 0, 0] * exp(
                -(1 / (2 * sigma_bruit**2)) * ecart**2
            )
        weights[k, :, 0, 0] /= sum(weights[k, :, 0, 0])
        x_mean[k, :, :] = np.sum(
            weights[k, :] * part_pred,
            axis=0,
        )
        cov_mats[k, :, :] = np.sum(
            weights[k, :]
            * np.matmul(
                part_pred - x_mean[k],
                np.transpose(part_pred - x_mean[k], axes=[0, 2, 1]),
                axes=[(-2, -1), (-2, -1), (-2, -1)],
            ),
            axis=0,
        )
        # Redistribution:
        if 1 / np.sum(weights[k, :, 0, 0] ** 2) < n_redis:
            redis_moments[k] = 1
            big_i = rd.multinomial(n=n_part, pvals=weights[k, :, 0, 0])
            index_new_part = 0  # pylint: disable=invalid-name
            for i in range(n_part):
                if big_i[i] != 0:
                    kern_dev = rd.multivariate_normal(
                        mean=6 * [0], cov=h_fin**2 * cov_mats[k], size=big_i[i]
                    ).reshape((big_i[i], 6, 1))
                    particles[k, index_new_part : index_new_part + big_i[i], :, :] = (
                        part_pred[i] + kern_dev
                    )  # pylint: disable=superfluous-parens
                    index_new_part += big_i[i]
            weights[k, :, 0, 0] = 1 / n_part
            x_mean[k, :, :] = np.sum(
                weights[k, :] * part_pred,
                axis=0,
            )
            cov_mats[k, :, :] = np.sum(
                weights[k, :]
                * np.matmul(
                    part_pred - x_mean[k],
                    np.transpose(part_pred - x_mean[k], axes=[0, 2, 1]),
                    axes=[(-2, -1), (-2, -1), (-2, -1)],
                ),
                axis=0,
            )
        else:
            particles[k, :, :, :] = part_pred[:, :, :]
        for i in range(n_part):
            part_pred[i, :, :] = phi @ particles[k, i, :, :]
        weight_pred[:, :, :] = weights[k, :, :, :]
    return x_mean, particles, cov_mats, redis_moments

In [ ]:
def step_viewer(
    x_val: np.ndarray,
    y_val: np.ndarray,
    h_mat: np.ndarray,
    particles: np.ndarray,
    etat_avion: np.ndarray,
) -> Callable[[int], None]:
    def view(k: int) -> None:
        fig = plt.figure(figsize=(15, 15))
        axe = fig.add_axes((0, 0, 1, 1))
        axe.set_ylim(200000, 140000)
        axe.set_xlim(140000, 200000)
        axe.pcolormesh(x_val, y_val, h_mat, zorder=-3)
        axe.plot(etat_avion[0, :], etat_avion[1, :], color="orange", zorder=-2)
        axe.scatter(
            particles[k, :, 0, 0],
            particles[k, :, 1, 0],
            color="pink",
            marker=".",
            s=0.5,
            zorder=-1,
        )
        axe.scatter(etat_avion[0, k], etat_avion[1, k], color="red", s=4, zorder=0)
        fig.savefig(
            join("data", f"particles_{k}.png"), format="png", bbox_inches="tight"
        )

    return view

In [ ]:
x_mean, particles, cov_mats, redis_moments = particle_filter(
    h_mes=h_mes,
    h_mnt=currified_h_mnt(x_val=x_val, y_val=y_val, h_mat=h_mat),
    init_particles=init_particles,
    phi=PHI,
    sigma_bruit=SIGMA_BRUIT,
)
see_step = step_viewer(
    x_val=x_val, y_val=y_val, h_mat=h_mat, particles=particles, etat_avion=etat_avion
)

In [ ]:
see_step(k=460)  # type: ignore

In [ ]:
def see_cov_axe(cov: np.ndarray, axis: int, name: str = " "):
    fig = plt.figure()
    axe = fig.add_axes((0, 0, 1, 1))
    axe.set_title(f"Ecart-type de ${name}$ au fil des iterations")
    axe.set_xlabel("iteration")
    axe.set_ylabel(f"$\\sigma({name})$")
    axis_sqrt = np.sqrt(cov[:, axis, axis])
    print(axis_sqrt[-1])
    axe.plot(axis_sqrt)
    fig.show()

In [ ]:
def cov_bilan(cov: np.ndarray):
    fig = plt.figure(figsize=(15, 15))
    axe = fig.add_axes((0, 0, 1, 1))
    axe.set_title("Ecart-types au fil des iterations")
    axe.set_xlabel("iteration")
    axe.set_ylabel("$\\sigma \\ [m]$")
    for axis, name in {0: "x", 1: "y", 2: "z"}.items():
        axis_sqrt = np.sqrt(cov[:, axis, axis])
        print(f"{name} : {axis_sqrt[-1]}")
        axe.plot(axis_sqrt, label=f"$\\sigma ({name})$")
    cov_trace = np.sqrt(np.trace(cov_mats[:, :3, :3], axis1=-2, axis2=-1))
    axe.plot(cov_trace, label="$\\sqrt{tr(\\text{Cov})}$")
    axe.legend(loc="upper right", shadow=True)
    fig.savefig(join("data", "sigmas.png"), format="png", bbox_inches="tight")
    fig.show()

In [ ]:
cov_bilan(cov=cov_mats)

In [ ]:
fig = plt.figure(figsize=(15, 15))
axe = fig.add_axes((0, 0, 1, 1))
axe.set_title("Redistribution au fil des iterations (redistribue si = 1)")
axe.set_xlabel("iteration")
axe.plot(redis_moments)
fig.savefig(join("data", "redis.png"), format="png", bbox_inches="tight")
fig.show()